*prep*
```cypher
// Create monopartite graph
MATCH (c:Client)-[:PERFORMED]->(t:Transaction)-[:TO]->(oc:Client)
with c, oc, sum(t.amount) AS weight
CREATE (c)-[:PAYS{amount: weight}]->(oc);
```

In [1]:
from neo4j import GraphDatabase
from gdsclient import Neo4jQueryRunner, GraphDataScience

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test1234"), encrypted=False)
runner = Neo4jQueryRunner(driver)
gds = GraphDataScience(runner)

In [3]:
def createGraphProjection(tx):
    return tx.run(
        """CALL gds.graph.create(
            'payment', 
            'Client', 
            { PAYS: { type: 'PAYS', orientation:'UNDIRECTED'}}, 
            {
                nodeProperties: { },
                relationshipProperties: {amount: { property: 'amount', aggregation: 'NONE', defaultValue: 0.0 }}
            }
        )"""
    ).consume()

In [4]:
def runFastRP(tx): 
    return tx.run(
        """CALL gds.fastRP.mutate(
            'payment', 
            {
                embeddingDimension: 8,
                iterationWeights: [0.0, 0.20, 0.40, 0.60, 0.80, 1.00, 1.20, 1.40, 1.60, 1.80, 2.00],
                relationshipWeightProperty: 'amount',
                mutateProperty:'fastRP'
            }
            )"""
    ).consume()    

In [5]:
def writeEmbedding(tx):
    return tx.run(
        """
        CALL gds.graph.writeNodeProperties('payment', ['fastRP'])
        """
    ).consume()

In [6]:
def runKNN(tx):
    return tx.run(
        """
        CALL gds.beta.knn.write('payment', 
        {
            writeRelationshipType: 'SIMILAR_EMBEDDING',
            writeProperty: 'score',
            topK: 5,
            randomSeed: 42,
            nodeWeightProperty: 'fastRP'
        }) YIELD nodesCompared, relationshipsWritten
        """
    ).consume()

In [7]:
def dropGraphProjection(tx):
    return tx.run("CALL gds.graph.drop('payment')").consume()

In [10]:
# Create projection 
#with driver.session(database = "paysim") as session:
#    result = session.write_transaction(createGraphProjection)
#    print(result)
graph = gds.graph.create(
    graph_name='purchases', 
    node_spec=['Client'], 
    relationship_spec={ 'PAYS': { 'properties': {'amount': { 'property': 'amount', 'aggregation': 'NONE', 'defaultValue': 0.0 }} , 'orientation':'UNDIRECTED'}}
)
print(f"Graph named {graph.name} created")


AttributeError: 'Result' object has no attribute 'data'

In [ ]:
# Calculate embedding (mutate)
with driver.session(database = "paysim") as session:
    result = session.write_transaction(runFastRP)
    print(result)

In [ ]:
# KNN + write back relationships
with driver.session(database = "paysim") as session:
    result = session.write_transaction(runKNN)
    print(result)

In [ ]:
# Write back embedding
with driver.session(database = "paysim") as session:
    result = session.write_transaction(writeEmbedding)
    print(result)

In [ ]:
# Drop projection
with driver.session(database = "paysim") as session:
    result = session.read_transaction(dropGraphProjection)
    print(result)

*demo query*
```cypher
//Similar demo
match p=(m:Mule)-[:SIMILAR_EMBEDDING]-(c:Client)
WITH p,m,c skip 1 limit 1
call {
  WITH m
  MATCH mp=(m)-[:PAYS*1..2]-()
  RETURN collect(mp)[0..100] as mp
}
call {
  WITH c
  MATCH cp=(c)-[:PAYS*1..2]-() 
  RETURN collect(cp)[0..100] as cp
}
RETURN p, mp, cp
```

*clean up*
```cypher
//Remove relationship produced by knn
match ()-[r:SIMILAR_EMBEDDING]-() delete r
```